In [1]:
#import statements
import azure.core
from azureml.core import Workspace,Environment,Experiment,ScriptRunConfig
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

In [2]:
# Naming Cluster and Compute-Target
aml_compute_target = "automldemo1" #ALL SMALL LETTER, NO underscore, 16ws long only.
experiment_name= 'automldemo1'

# Workspace

In [3]:
#Getting workspace access
ws=Workspace.from_config()

In [4]:
#Displaying the Workspace
ws

Workspace.create(name='charanml1', subscription_id='7ab476c7-a184-4dee-865f-f9d09e32023e', resource_group='charan')

# Compute Instance

In [5]:
#Creating or Using Exisiting Compute Instance 
import datetime
import time

from azureml.core.compute import ComputeTarget, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your instance
# Compute instance name should be unique across the azure region
compute_name = "automldemo1"

# Verify that instance does not exist already
try:
    instance = ComputeInstance(workspace=ws, name=compute_name)
    print('Found existing instance, use it.')
except ComputeTargetException:
    compute_config = ComputeInstance.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        ssh_public_access=False,
        # vnet_resourcegroup_name='<my-resource-group>',
        # vnet_name='<my-vnet-name>',
        # subnet_name='default',
        # admin_user_ssh_public_key='<my-sshkey>'
    )
    instance = ComputeInstance.create(ws, compute_name, compute_config)
    instance.wait_for_completion(show_output=True)

Found existing instance, use it.


We can use Computer cluster Or Compute Instance to train the model 
```
# Create Compute Target
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("This Compute Target already exist.")
except ComputeTargetException:
    print("creating new compute target :",aml_compute_target)
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4,
                                               idle_seconds_before_scaledown=3000)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached now")

 
```

# Experiment

In [6]:
# Creating and display Experiment
exp = Experiment(ws,experiment_name)
exp

Name,Workspace,Report Page,Docs Page
automldemo1,charanml1,Link to Azure Machine Learning studio,Link to Documentation


# Environment

In [7]:
#Creating the Environment and yml file for training and deployment 
from azureml.core.conda_dependencies import CondaDependencies 
import platform

# Usually a good idea to choose specific version numbers
# so training is made on same packages as scoring
myenv = CondaDependencies.create(conda_packages=['python==3.6.2','numpy',
                            'scikit-learn==0.23.2', 'pandas'],
                             pip_packages = ['applicationinsights','azureml-defaults', 'debugpy','joblib'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

#deployment of a registered model

from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="Session33-env", file_path="./myenv.yml")

    

We are ready with environment and expriment we need to create tarining script 

```
% mytrain_log.py
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,auc
import pandas as pd
import joblib
import argparse

from azureml.core import Run

run = Run.get_context()

data=pd.read_csv("./diabetes.csv")
print(data.columns)

X=data.iloc[:,:-1]
y=data.iloc[:,-1]

trainx,testx,trainy,testy=train_test_split(X, y, random_state=101, test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=0)
model = rf.fit(trainx, trainy)
run.log("Parameter",rf.get_params())
print("Accuracy on training set: {:.3f}".format(rf.score(trainx, trainy)))
print("Accuracy on test set: {:.3f}".format(rf.score(testx, testy)))

y_pred=model.predict(testx)
classification_report(testy,y_pred)
confusion_matrix(testy,y_pred)

acc=accuracy_score(testy,y_pred)
run.log("Accuracy",acc)
run.log("Fit", model)


print(accuracy_score(testy,y_pred))
joblib.dump(model, "outputs/diabeticmodel.pkl")
 
```

In [8]:
%%writefile mytrain_log.py
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,auc
import pandas as pd
import joblib
import argparse

from azureml.core import Run

run = Run.get_context()

data=pd.read_csv("./diabetes.csv")
print(data.columns)

X=data.iloc[:,:-1]
y=data.iloc[:,-1]

trainx,testx,trainy,testy=train_test_split(X, y, random_state=101, test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=0)
model = rf.fit(trainx, trainy)
run.log("Parameter",rf.get_params())
print("Accuracy on training set: {:.3f}".format(rf.score(trainx, trainy)))
print("Accuracy on test set: {:.3f}".format(rf.score(testx, testy)))

y_pred=model.predict(testx)
classification_report(testy,y_pred)
confusion_matrix(testy,y_pred)

acc=accuracy_score(testy,y_pred)
run.log("Accuracy",acc)
run.log("Fit", model)


print(accuracy_score(testy,y_pred))
joblib.dump(model, "outputs/diabeticmodel.pkl")

Writing mytrain_log.py


# Submit Run to Experement 

In [9]:
# Create environment to execute your code on Cloud
# env = Environment.from_pip_requirements(name="mlops_v2",file_path="Requirement.txt")
config=ScriptRunConfig(source_directory=".",script="mytrain_log.py",compute_target=instance,environment=myenv,
)
execution=exp.submit(config)
execution.wait_for_completion(show_output=True)


RunId: automldemo1_1669030589_a5b18017
Web View: https://ml.azure.com/runs/automldemo1_1669030589_a5b18017?wsid=/subscriptions/7ab476c7-a184-4dee-865f-f9d09e32023e/resourcegroups/charan/workspaces/charanml1&tid=33d8a930-f93a-4f9c-a9b7-e4be5012bfb9

Streaming azureml-logs/20_image_build_log.txt

2022/11/21 11:35:49 Downloading source code...
2022/11/21 11:35:50 Finished downloading source code
2022/11/21 11:35:50 Creating Docker network: acb_default_network, driver: 'bridge'
2022/11/21 11:35:51 Successfully set up Docker network: acb_default_network
2022/11/21 11:35:51 Setting up Docker configuration...
2022/11/21 11:35:51 Successfully set up Docker configuration
2022/11/21 11:35:51 Logging in to registry: 98894efc48c9422ba7cd0a7185246bf6.azurecr.io
2022/11/21 11:35:52 Successfully logged into 98894efc48c9422ba7cd0a7185246bf6.azurecr.io
2022/11/21 11:35:52 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2022/11/21 11:35:52 Scannin

{'runId': 'automldemo1_1669030589_a5b18017',
 'target': 'automldemo1',
 'status': 'Completed',
 'startTimeUtc': '2022-11-21T11:46:06.093047Z',
 'endTimeUtc': '2022-11-21T11:47:12.715382Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '07137adf-d802-4bdd-9654-d78282b1bdbe',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'mytrain_log.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'automldemo1',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': None,
  'environment': {'name': 'Session33-en

In [10]:
execution.get_environment()

{
    "assetId": "azureml://locations/eastus/workspaces/98894efc-48c9-422b-a7cd-0a7185246bf6/environments/Session33-env/versions/Autosave_2022-11-21T11:35:46Z_42c9b5f4",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20221010.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,


In [11]:
#Download the model created by the expriment Run
from azureml.core import Workspace, Experiment, Run
print(execution)


# get workspace
ws = Workspace.from_config()

# get/create experiment
exp = Experiment(ws, experiment_name)

# get run
run_exp = Run(exp, execution.id)

# printing log matrics
print(run_exp.get_metrics())

# Download model file from the JOb Run

run_exp.download_file("outputs/diabeticmodel.pkl")

Run(Experiment: automldemo1,
Id: automldemo1_1669030589_a5b18017,
Type: azureml.scriptrun,
Status: Completed)
{'Parameter': "{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 0, 'verbose': 0, 'warm_start': False}", 'Accuracy': 0.7662337662337663, 'Fit': 'RandomForestClassifier(random_state=0)'}


# Model Registration

In [12]:
#Register the model to workspace 

#import sklearn
import os
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=ws,
                       # Name of the registered model in your workspace.
                       model_name='automldemo1',
                       # Local file to upload and register as a model.
                       model_path='./diabeticmodel.pkl',
                       # Framework used to create the model.
                       model_framework=Model.Framework.SCIKITLEARN,
                       # Version of scikit-learn used to create the model.
                       model_framework_version="1.0.1",
                       resource_configuration=ResourceConfiguration(
                           cpu=1, memory_in_gb=0.5),
                       description='Random Forest for Diabetes Classification',
                       tags={'area': 'diabetes', 'type': 'CART'})

print('Name:', model.name)
print('Version:', model.version)
model.download(target_dir=os.getcwd(), exist_ok=True)

Registering model automldemo1
Name: automldemo1
Version: 3


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/cinst1/code/Users/Charana.Umesha/Azure_ml_training_session3/diabeticmodel.pkl'

# Model Deployment  Without Code 

In [13]:
# Deploying the model Without code 
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration
service = Model.deploy(ws,"automldemo1", #This is endpoint name
                           models=[model],
                           overwrite=True)
service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

/tmp/ipykernel_65824/1512466389.py:4: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws,"automldemo1", #This is endpoint name


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-21 11:47:21+00:00 Creating Container Registry if not exists.
2022-11-21 11:47:21+00:00 Registering the environment.
2022-11-21 11:47:22+00:00 Uploading autogenerated assets for no-code-deployment.
2022-11-21 11:47:22+00:00 Use the existing image.
2022-11-21 11:47:23+00:00 Generating deployment configuration.
2022-11-21 11:47:23+00:00 Submitting deployment to compute.
2022-11-21 11:47:26+00:00 Checking the status of deployment automldemo1..
2022-11-21 11:52:51+00:00 Checking the status of inference endpoint automldemo1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://7d288be9-c493-4169-b878-cf30b207c214.eastus.azurecontainer.io/score


In [14]:
print(service.get_logs())

2022-11-21T11:52:27,991426055+00:00 - iot-server/run 
2022-11-21T11:52:27,991290255+00:00 - rsyslog/run 
2022-11-21T11:52:27,993293955+00:00 - gunicorn/run 
2022-11-21T11:52:27,995713955+00:00 | gunicorn/run | 
2022-11-21T11:52:27,998241255+00:00 | gunicorn/run | ###############################################
2022-11-21T11:52:28,000651055+00:00 | gunicorn/run | AzureML Container Runtime Information
2022-11-21T11:52:28,003054655+00:00 | gunicorn/run | ###############################################
2022-11-21T11:52:28,006358755+00:00 | gunicorn/run | 
2022-11-21T11:52:28,009420355+00:00 | gunicorn/run | 
2022-11-21T11:52:28,040437255+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20221101.v11
2022-11-21T11:52:28,043950155+00:00 | gunicorn/run | 
2022-11-21T11:52:28,051031755+00:00 | gunicorn/run | 
2022-11-21T11:52:28,052474955+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_735bc617fbe6d8e5b40a363cb1083ec4/bin:/

# Testing the deployed Model

In [15]:
# Prepare your test data
import pandas as pd
import json
data = pd.read_csv('test.csv')
xt = data[:6].values.tolist()
test = json.dumps({"data":xt})
test

'{"data": [[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0], [1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0], [8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0], [1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0], [0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0], [5.0, 116.0, 74.0, 0.0, 0.0, 25.6, 0.201, 30.0]]}'

In [16]:
#Testing the Deployed endpoint
from azureml.core import Webservice
service = Webservice(workspace=ws, name="automldemo1")
print(service.scoring_uri)
print(service.swagger_uri)
prediction = service.run(input_data=test)
prediction
# http://756dbf5d-b758-48d3-ac85-93fb969239ae.eastus.azurecontainer.io/score
# http://756dbf5d-b758-48d3-ac85-93fb969239ae.eastus.azurecontainer.io/swagger.json

http://7d288be9-c493-4169-b878-cf30b207c214.eastus.azurecontainer.io/score
http://7d288be9-c493-4169-b878-cf30b207c214.eastus.azurecontainer.io/swagger.json


{'predict_proba': [[0.11, 0.89],
  [0.97, 0.03],
  [0.5, 0.5],
  [1.0, 0.0],
  [0.17, 0.83],
  [0.93, 0.07]]}

In [17]:
xt = data.values.tolist()
xt[0:2]
tworecords = json.dumps({"data":xt[0:2]})
tworecords

'{"data": [[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0], [1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0]]}'

In [18]:
prediction = service.run(input_data=tworecords)
prediction

{'predict_proba': [[0.11, 0.89], [0.97, 0.03]]}

In [19]:
# Real time scoring using Rest Call
import requests
headers = {'content-Type':'application/json'}

response = requests.post(service.scoring_uri, test, headers=headers)
response.text

'{"predict_proba": [[0.11, 0.89], [0.97, 0.03], [0.5, 0.5], [1.0, 0.0], [0.17, 0.83], [0.93, 0.07]]}'

```
data=[
    {
      "0": 6,
      "1": 148,
      "2": 72,
      "3": 35,
      "4": 0,
      "5": 33.6,
      "6": 0.627,
      "7": 50,
    }
  ]
import pandas as pd
pd.DataFrame(data)
```

# Model Deployment with score.py

In [20]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script='score.py',source_directory='.', environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 2, 
                                               memory_gb = 4, 
                                               enable_app_insights=True,
                                               description = 'Iris classification service')

aci_service_name = 'session-33-withcode-deployment'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig,overwrite=True)
aci_service.wait_for_deployment(True)
print(aci_service.state)

session-33-withcode-deployment


/tmp/ipykernel_65824/3608158481.py:16: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig,overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-11-21 11:53:05+00:00 Creating Container Registry if not exists.
2022-11-21 11:53:05+00:00 Registering the environment.
2022-11-21 11:53:06+00:00 Use the existing image.
2022-11-21 11:53:06+00:00 Generating deployment configuration.
2022-11-21 11:53:07+00:00 Submitting deployment to compute.
2022-11-21 11:53:11+00:00 Checking the status of deployment session-33-withcode-deployment.